In [1]:
import sys
import os
import csv

import numpy as np
from matplotlib import pyplot as plt
from __future__ import print_function

In [6]:
# Create the class for entry objects
class ENTRY:
    
    def __init__(self, entry, authors, quantity, element, density, temperature, comments, data):
        self.entry       = entry
        self.authors     = authors
        self.quantity    = quantity
        self.element     = element
        self.density     = density
        self.temperature = temperature
        self.comments    = comments
        self.data        = data

    def show_entry(self, displayCom):
        print('entry ', self.entry)
        print('    Element:', self.element, '  -  ', self.quantity)
        print('    Density (g/cc) = ', self.density)
        print('    Temperature (eV) = ', self.temperature)
        if displayCom == True:
            for com in self.comments:
                print('   ', com)
        print('')

        
        
        
# Create the class for database objects
class DATABASE(dict):
        
    def search(self, filterMethod, value, displayCom):
        if 'entry' == filterMethod:
            for key in self:
                if self[key].entry == value:
                    self[key].show_entry(displayCom)
                    
        elif 'author' == filterMethod:
            for key in self:
                if(any(author == value for author in self[key].authors)):
                    print("authors: ", self[key].authors)
                    self[key].show_entry(displayCom)

        elif 'quantity' == filterMethod:
            for key in self:
                if self[key].quantity == value:
                    self[key].show_entry(displayCom)
                    
        elif 'element' == filterMethod:
            for key in self:
                if self[key].element == value:
                    self[key].show_entry(displayCom)
                    
        else:
            print('Search method not found.')

                
    def plot(self, entries, colors):
        n = 0
        for key in entries:
            plt.plot(database[key].data[:,0], database[key].data[:,1], colors[n])
            n = n+1
        plt.show()

In [ ]:
                            #  INITIALIZE THE DATABASE

showDatabase = True
#showDatabase = False

#debugMode = True
debugMode = False


entryNb = 0
database = DATABASE()


mainDir = os.getcwd()
elm_Inside_MainDir = os.listdir(mainDir)

for element in elm_Inside_MainDir:
    if(element[0] == "."): 
        continue
    
    if(os.path.isdir(element)):
        if(showDatabase): 
            print(element)
        
        subDir = element
        data_Files = os.listdir(subDir)
        for datFile in data_Files:
            if(datFile[0] == "."): 
                continue
            if(showDatabase): 
                print("    ", datFile)
            
            
            # open csv file
            csvfile = open(mainDir+"/"+subDir+"/"+datFile, "rb")
            # read csv file according to dialect
            reader = csv.reader( csvfile, delimiter=',' )
            
            
            # Create the entry name
            tempSubDir = subDir.split('_')
            tempDatFile = datFile.split('.')
            entryName = tempSubDir[1]+tempSubDir[0]+'_'+tempDatFile[0]
            if any(database[key].entry == entryName for key in database):
                print('ERROR: duplicated entryName in database.')
                print('EXITING')
                sys.exit()
            if debugMode:
                print("       ","Entry name:", entryName)
                
            
            # read header part of the file
            h0 = "#"
            while (h0 == "#"):
                header = next(reader, None)
                if(len(header) == 0):
                    h0 = ''
                    continue
                h0 = header[0][0]
                tempList = header[0].split('=')
                tempList[0] = tempList[0].strip()
                
                
                if "# Authors" == tempList[0]:
                    header[0] = (header[0].split("="))[1]
                    authors = []
                    for a in header:
                        a = a.strip()
                        authors = authors +[a,]
                    if debugMode == True:
                        print('        authors = ', authors)
                
                elif "# Quantity" == tempList[0]:
                    quantity = tempList[1].strip()
                    if debugMode == True:
                        print('        quantity = ', quantity)
                        
                elif "# Element" == tempList[0]:
                    element = tempList[1].strip()
                    if debugMode == True:
                        print('        element = ', element)
                
                elif "# Density (g/cc)" == tempList[0]:
                    density = tempList[1].strip()
                    if debugMode == True:
                        print('        density = ', density)
                
                elif "# Temperature (eV)" == tempList[0]:
                    temperature = tempList[1].strip()
                    if debugMode == True:
                        print('        temperature = ', temperature)
                        
                elif "# Comments" == tempList[0]:
                    comments = []
                    header = next(reader, None)
                    h0 = header[0][0]
                    while h0 == "#":
                        if len(header) > 1:
                         header = [','.join(header)]
                        
                        comments = comments +header
                        header = next(reader, None)
                        if len(header) == 0:
                         h0 = ''
                         continue
                        h0 = header[0][0]
                    if debugMode == True:
                        for com in comments:
                         print('       ', com)
                            
                else:
                    print('ERROR: Unrecognized parameter in header')
                    print('EXITING')
                    sys.exit()
                    
                
            # Get the data curve
            dataList = [row for row in reader]
            data = np.asarray(dataList)  
            
            # close the CSV file
            csvfile.close()
                
            # Insert the new entry into the database
            newEntry = ENTRY(entryName, authors, quantity, element, density, temperature, comments, data)
            database[entryName] = newEntry
            if(debugMode == True):
                print('database size = ', len(database))
                print('')
            

            
print('Size of the database = ', len(database), 'entries'

In [ ]:
# Get information from database

print 'database[Clerouin2016_pcf_W_400eV].entry = ',       database['Clerouin2016_pcf_W_400eV'].entry
print 'database[Clerouin2016_pcf_W_400eV].authors = ',     database['Clerouin2016_pcf_W_400eV'].authors
print 'database[Clerouin2016_pcf_W_400eV].quantity = ',    database['Clerouin2016_pcf_W_400eV'].quantity
print 'database[Clerouin2016_pcf_W_400eV].element = ',     database['Clerouin2016_pcf_W_400eV'].element
print 'database[Clerouin2016_pcf_W_400eV].density = ',     database['Clerouin2016_pcf_W_400eV'].density
print 'database[Clerouin2016_pcf_W_400eV].temperature = ', database['Clerouin2016_pcf_W_400eV'].temperature



In [ ]:
# Search inside the database

#display_comments = True
display_comments = False


#database.search('entry', 'Clerouin2016_pcf_W_400eV', display_comments)

database.search('author', 'Pam', display_comments)

#database.search('quantity', 'PCF', display_comments)

#database.search('element', 'Fe', display_comments)



In [ ]:
# Print data

entryList = ['Clerouin2015_pcf_Ag_200eV', 
             'Clerouin2015_pcf_W_100eV',
             'Clerouin2016_pcf_W_400eV']
styles = ['k','r-','b--']


database.plot(entryList, styles)

